In [ ]:
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.optim import lr_scheduler

from utils import train # custom function to train the model

# For evaluation metrics
from sklearn.metrics import (
    f1_score, recall_score, precision_score,
    confusion_matrix, classification_report
)

In [27]:
# Check device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


<h1 style="color: orange;">Data Augmentation & Processing</h1>

In [28]:
current_path = os.getcwd()
current_path = os.path.dirname(current_path)
data_dir = os.path.join(current_path, "3-Train-Val-Test-Split")

In [29]:
input_size = 224 # ResNet50 expects 224x224 images

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),  # Random cropping and resizing
        transforms.RandomHorizontalFlip(p=0.5),  # Flip images horizontally with 50% probability
        transforms.RandomVerticalFlip(p=0.5),  # Flip images vertically with 50% probability (if plausible)
        transforms.RandomRotation(degrees=45),  # Rotate images randomly within ±45 degrees
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, etc.
        transforms.RandomAffine(
            degrees=15,  # Rotate within ±15 degrees
            translate=(0.1, 0.1),  # Allow small translations (10% of image size)
            scale=(0.9, 1.1),  # Allow small zoom in/out
            shear=10  # Allow small shearing
        ),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # Apply slight blurring
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize for ResNet50
    ]),
    'val': transforms.Compose([
        transforms.Resize(int(input_size * 1.1)),  # Resize slightly larger to allow for CenterCrop
        transforms.CenterCrop(input_size),  # Crop to the target size
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(int(input_size * 1.1)),  # Resize slightly larger to allow for CenterCrop
        transforms.CenterCrop(input_size),  # Crop to the target size
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


<h1 style="color: orange;">Data Loading</h1>

In [30]:
image_datasets = {
    phase: datasets.ImageFolder(
        os.path.join(data_dir, phase), 
        transform=data_transforms[phase]
    )
    for phase in ['train', 'val', 'test']
}

In [31]:
# Create data loaders
batch_size = 64
dataloaders = {
    phase: DataLoader(
        image_datasets[phase], 
        batch_size=batch_size, 
        shuffle=True if phase == 'train' else False,
        num_workers=2
    )
    for phase in ['train', 'val', 'test']
}

In [32]:
# Dataset sizes and class names
dataset_sizes = {phase: len(image_datasets[phase]) for phase in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes
print("Classes:", class_names)
print("Dataset sizes:", dataset_sizes)

Classes: ['asteroid', 'comet', 'galaxy', 'nebula', 'planet', 'star']
Dataset sizes: {'train': 1800, 'val': 600, 'test': 600}


<h1 style="color: orange;">Hyperparameters</h1>

In [33]:
scheduler_step_size = 7
scheduler_gamma = 0.1

<h1 style="color: orange;">Transfer Learning</h1>

In [34]:
model = models.resnet50(weights="ResNet50_Weights.DEFAULT")

In [35]:
# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False

In [36]:
# Modify the final layer to match the number of classes
num_ftrs = model.fc.in_features
num_classes = len(class_names)  # Number of classes in your dataset
model.fc = nn.Linear(num_ftrs, num_classes)  # num_classes = 6 (as per dataset)
model = model.to(device) # Move model to device (GPU or CPU)

<h2 style="color: orange;">Hyperparameters</h2>

In [38]:
criterion = nn.CrossEntropyLoss() # Loss function
learning_rate = 1e-3 
epochs = 20
scheduler_step_size = 7
scheduler_gamma = 0.1
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate) # Optimizer
scheduler = lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)

In [39]:
# Train the model for a few epochs with early stopping
model, history = train_model(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=5,  # Reduced epochs for quick validation
    patience=2,  # Stop early if no improvement in 2 epochs
    min_delta=0.01  # Minimum improvement to consider
)

NameError: name 'train_model' is not defined

In [ ]:
plot_training_history(history)